In [8]:
import spiceypy
import datetime
import numpy as np

In [9]:
spiceypy.furnsh("../kernels/lsk/naif0012.tls")
spiceypy.furnsh("../kernels/spk/de432s.bsp")
spiceypy.furnsh("../kernels/pck/gm_de431.tpc")

In [10]:
date_today = datetime.datetime.today()
date_today_str = date_today.strftime("%Y-%m-%dT00:00:00")
et_today_midnight = spiceypy.utc2et(date_today_str)
earth_state_wrt_sun, earth_sun_light_time = spiceypy.spkgeo(
    targ=399, et=et_today_midnight, ref="ECLIPJ2000", obs=10
)

In [11]:
# Create a numpy array of the Earth location information
earth_state_wrt_sun = np.array(earth_state_wrt_sun)
# Compute the distance of the Earth from the sun (km)
earth_sun_distance = np.linalg.norm(earth_state_wrt_sun[:3])
# Earth orbit velocity (km/s)
earth_orb_speed_wrt_sun = np.linalg.norm(earth_state_wrt_sun[3:])
earth_orb_speed_wrt_sun

30.246298829600995

In [12]:
# Extract an array with the mass times the Gravity constant of the sun (GM)
_, GM_SUN = spiceypy.bodvcd(bodyid=10, item="GM", maxn=1)
# Define a function to determine velocity of an orbiting body
v_orb_func = lambda gm, r: np.sqrt(gm / r)
# Define the theoretical velocity of a body orbiting the sun
earth_orbit_speed_speed_wrt_sun_theory = v_orb_func(gm=GM_SUN[0], r=earth_sun_distance)
earth_orbit_speed_speed_wrt_sun_theory

30.01925218091564

In [13]:
# Array of Earth's position values
earth_position_wrt_sun = earth_state_wrt_sun[:3]
# Convert distance values to AU
earth_position_wrt_sun_normed = earth_position_wrt_sun / earth_sun_distance
# Create array of position at autumnal equinox, x=1, y,z=0
earth_position_wrt_sun_normed_autumn = np.array([1.0, 0.0, 0.0])

In [14]:
# Compute the current angular distance from the Earth's location at the autumnal equinox
ang_dist_deg = np.degrees(
    np.arccos(
        np.dot(earth_position_wrt_sun_normed, earth_position_wrt_sun_normed_autumn)
    )
)
ang_dist_deg

81.63926269232316